In [ ]:
from Lib.myModules.my_ML_Modules_1 import *
import math
import sys
import contextlib

In [ ]:
data_dir = "../data"
infantNum = 6
ECG_dataset = f"{data_dir}/infant{infantNum}_ecg"
RESP_dataset = f"{data_dir}/infant{infantNum}_resp"
# Read all the data
signal_ecg_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_ecg")
signal_resp_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_resp")
print(f'ECG DATA: {signal_ecg_0[1]}')
print(F'RESP DATA: {signal_resp_0[1]}')
totalDuration_ECG = num2Time(num=signal_ecg_0[1]['sig_len'], fs=signal_ecg_0[1]['fs'])
print(f'{totalDuration_ECG} sec, {secs2minutes(totalDuration_ECG)} minutes, {secs2hours(totalDuration_ECG)} hours')
totalDuration_RESP = num2Time(num=signal_resp_0[1]['sig_len'], fs=signal_resp_0[1]['fs'])
print(f'{totalDuration_RESP} sec, {secs2minutes(totalDuration_RESP)} minutes, {secs2hours(totalDuration_RESP)} hours')
# Start and end time defined
# start_time_secs = hours2secs(1)
# end_time_secs = hours2secs(1.1)

def training_pipeline(start_time_secs, end_time_secs):
    ECG_startNum = time2Num(start_time_secs, signal_ecg_0[1]['fs'])
    ECG_endNum = time2Num(end_time_secs, signal_ecg_0[1]['fs'])
    print(f'start: {ECG_startNum}, end: {ECG_endNum}')
    RESP_startNum = time2Num(start_time_secs, signal_resp_0[1]['fs'])
    RESP_endNum = time2Num(end_time_secs, signal_resp_0[1]['fs'])
    print(f'start: {RESP_startNum}, end: {RESP_endNum}')
    signal_ECG, signal_ECG_startTime_secs, signal_ECG_endTime_secs = read_Data(ECG_dataset, startNum=ECG_startNum, endNum=ECG_endNum)
    signal_RESP, signal_RESP_startTime_secs, signal_RESP_endTime_secs = read_Data(RESP_dataset, startNum=RESP_startNum, endNum=RESP_endNum)
    # print(signal_ECG[0].shape)
    signal_ECG_1 = iqr_smooth(signal=signal_ECG[0],hiPerc=90,loPerc=10, cutoff_factor=1)
    # print(signal_RESP[0].shape)
    signal_RESP_1 = iqr_smooth(signal=signal_RESP[0],hiPerc=90,loPerc=10, cutoff_factor=1)
    signal_ECG_2 = norm_signal(signal=signal_ECG_1)
    signal_RESP_2 = norm_signal(signal=signal_RESP_1)
    hrs_0 = cal_heartrate(signal=signal_ECG_2, fs=signal_ECG[1]['fs'])
    rrs_0 = cal_resprate(signal=signal_RESP_2, fs=signal_RESP[1]['fs'])
    if (rrs_0.shape[0] < hrs_0.shape[0]):
        hrs_1 = data_resample(data_fixNan(hrs_0), rrs_0.shape[0])
        rrs_1 = data_fixNan(rrs_0)
    else:
        rrs_1 = data_resample(data_fixNan(rrs_0), hrs_0.shape[0])
        hrs_1 = data_fixNan(hrs_0)
    correlationTest(rrs_1[:],hrs_1[:], plot=False)
    print("---------------------------------------------")
    print("--- Linear Regression ---")
    lr_model_1 = train_lr_model(X=rrs_1, y=hrs_1)
    print("---------------------------------------------")
    print("--- Polynomial Regression ---")
    pr_model_1 = train_pr_model(X=rrs_1, y=hrs_1, degree=6)
    print("---------------------------------------------")
    print("--- SVR Radius Basis Function---")
    svr_rbf_1 = train_svr_rbf_model(X=rrs_1, y=hrs_1, param_C=100, param_gamma=0.1, seed=42)
    print("--- SVR Linear ---")
    svr_lin_1 = train_svr_lin_model(X=rrs_1, y=hrs_1, param_C=1000, param_gamma='auto', seed=42)
    print("--- SVR Polynomial ---")
    svr_poly_1 = train_svr_poly_model(X=rrs_1, y=hrs_1, param_C=1000, param_gamma='auto', param_degree=6, seed=42)
    return lr_model_1, pr_model_1, svr_rbf_1, svr_lin_1, svr_poly_1


def cascade_training_pipeline(lr_model, pr_model, svr_rbf_model, svr_lin_model, svr_poly_model, start_time_secs, end_time_secs):
    ECG_startNum = time2Num(start_time_secs, signal_ecg_0[1]['fs'])
    ECG_endNum = time2Num(end_time_secs, signal_ecg_0[1]['fs'])
    print(f'start: {ECG_startNum}, end: {ECG_endNum}')
    RESP_startNum = time2Num(start_time_secs, signal_resp_0[1]['fs'])
    RESP_endNum = time2Num(end_time_secs, signal_resp_0[1]['fs'])
    print(f'start: {RESP_startNum}, end: {RESP_endNum}')
    signal_ECG, signal_ECG_startTime_secs, signal_ECG_endTime_secs = read_Data(ECG_dataset, startNum=ECG_startNum, endNum=ECG_endNum)
    signal_RESP, signal_RESP_startTime_secs, signal_RESP_endTime_secs = read_Data(RESP_dataset, startNum=RESP_startNum, endNum=RESP_endNum)
    # print(signal_ECG[0].shape)
    signal_ECG_1 = iqr_smooth(signal=signal_ECG[0],hiPerc=90,loPerc=10, cutoff_factor=1)
    # print(signal_RESP[0].shape)
    signal_RESP_1 = iqr_smooth(signal=signal_RESP[0],hiPerc=90,loPerc=10, cutoff_factor=1)
    signal_ECG_2 = norm_signal(signal=signal_ECG_1)
    signal_RESP_2 = norm_signal(signal=signal_RESP_1)
    hrs_0 = cal_heartrate(signal=signal_ECG_2, fs=signal_ECG[1]['fs'])
    rrs_0 = cal_resprate(signal=signal_RESP_2, fs=signal_RESP[1]['fs'])
    if (rrs_0.shape[0] < hrs_0.shape[0]):
        hrs_1 = data_resample(data_fixNan(hrs_0), rrs_0.shape[0])
        rrs_1 = data_fixNan(rrs_0)
    else:
        rrs_1 = data_resample(data_fixNan(rrs_0), hrs_0.shape[0])
        hrs_1 = data_fixNan(hrs_0)
    correlationTest(rrs_1[:],hrs_1[:], plot=False)
    print("---------------------------------------------")
    print("--- Linear Regression ---")
    lr_model = cascade_train_lr_model(model=lr_model, X=rrs_1, y=hrs_1)
    print("---------------------------------------------")
    print("--- Polynomial Regression ---")
    pr_model = cascade_train_pr_model(model=pr_model, X=rrs_1, y=hrs_1, degree=6)
    print("---------------------------------------------")
    print("--- SVR Radius Basis Function---")
    svr_rbf_model = cascade_train_svr_rbf_model(model=svr_rbf_model, X=rrs_1, y=hrs_1, param_C=100, param_gamma=0.1)
    print("--- SVR Linear ---")
    svr_lin_model = cascade_train_svr_lin_model(model=svr_lin_model, X=rrs_1, y=hrs_1, param_C=1000, param_gamma='auto')
    print("--- SVR Polynomial ---")
    svr_poly_model = cascade_train_svr_poly_model(model=svr_poly_model, X=rrs_1, y=hrs_1, param_C=1000, param_gamma='auto', param_degree=6)
    return lr_model, pr_model, svr_rbf_model, svr_lin_model, svr_poly_model



In [ ]:
# initialize the training
start_time_secs = hours2secs(1)
end_time_secs = hours2secs(1.1)

lr_model, pr_model, svr_rbf_model, svr_lin_model, svr_poly_model = training_pipeline(start_time_secs=start_time_secs, end_time_secs=end_time_secs)

In [ ]:
logFilePath = f"../logs/infant{infantNum}_train_output.log"
with open(logFilePath, 'a') as f:
    with contextlib.redirect_stdout(f):
        for i in np.arange(1, math.floor(secs2hours(totalDuration_ECG)), 0.1):
            print("-------------------------------------------------------------------------")
            print(f"Data bin index: {i:.1f}")
            start_time_secs = hours2secs(i)
            end_time_secs = hours2secs(i+0.1)
            lr_model, pr_model, svr_rbf_model, svr_lin_model, svr_poly_model = cascade_training_pipeline(lr_model, pr_model, svr_rbf_model, svr_lin_model, svr_poly_model, start_time_secs=start_time_secs, end_time_secs=end_time_secs)

In [ ]:
# save the model
for i in [lr_model, pr_model, svr_rbf_model, svr_lin_model, svr_poly_model]:
    filename = f"../models/infant_{infantNum}/{i}.sav"
    joblib.dump(i, filename)
